---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle


---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def degree_distribution(G):
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i) / float(nx.number_of_nodes(G))
                for i in degree_values]
    return histogram

def graph_identification():
    results = []
    for G in P1_Graphs:
        cluster = nx.average_clustering(G)
        shortest_path = nx.average_shortest_path_length(G)
        degree_dist = degree_distribution(G)
        
        if len(degree_dist)>10:
            results.append('PA')
        elif cluster < 0.1:
            results.append('SW_H')
        else:
            results.append('SW_L')
    
    return results

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [8]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))


Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [24]:
def salary_predictions():
    from sklearn.model_selection import train_test_split 
    
    #     Question 3 assignment 3 applied text
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.metrics import roc_auc_score
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.ensemble import RandomForestClassifier


    #     Question 3 assignment 3 applied text
    
    df = pd.DataFrame(index=G.nodes())
    
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
#     https://datatofish.com/rows-with-nan-pandas-dataframe/
   
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
#     df['closeness'] = pd.Series(nx.closeness_centrality(G))
    
#     df['betweenness'] = pd.Series(nx.betweenness_centrality(G, normalized=True, endpoints=False))

    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['centrality'] = pd.Series(nx.degree_centrality(G))
    
    df_test = df[df['ManagementSalary'].isnull()]
    
    indexes_to_drop = df_test.index.tolist()
    
    df_train = df.drop(indexes_to_drop)
    
    y_train = df_train['ManagementSalary']  
    
    
    X_train = df_train.drop(['ManagementSalary'], axis=1)
    X_test =  df_test.fillna(0)
    X_test = X_test.drop(['ManagementSalary'], axis=1)
    clf = RandomForestClassifier().fit(X_train, y_train)
        


    probs =  clf.predict_proba(X_test)[:, 1]
    result = pd.Series(probs, index=X_test.index)
    
# AUC Score: 9070845440708455
    
    return result
salary_predictions()

1       0.0
2       0.9
5       1.0
8       0.4
14      0.3
18      0.5
27      0.0
30      0.8
31      0.9
34      0.0
37      0.4
40      0.0
45      0.0
54      0.7
55      0.6
60      1.0
62      1.0
65      0.9
77      0.0
79      0.0
97      0.0
101     0.0
103     0.9
108     0.1
113     0.5
122     0.1
141     0.1
142     1.0
144     0.0
145     0.6
       ... 
913     0.0
914     0.0
915     0.0
918     0.2
923     0.0
926     0.1
931     0.0
934     0.0
939     0.0
944     0.0
945     0.0
947     0.0
950     0.0
951     0.0
953     0.0
959     0.0
962     0.0
963     0.1
968     0.0
969     0.0
974     0.0
984     0.0
987     0.0
989     0.0
991     0.0
992     0.0
994     0.0
996     0.0
1000    0.0
1001    0.0
Length: 252, dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [27]:
def new_connections_predictions():
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.ensemble import RandomForestClassifier
    df2 = pd.DataFrame(future_connections)
    
    df2['Preferential Attachment'] = [i[2] for i in nx.preferential_attachment(G, df2.index)]
    
    df2['Common Neighbors'] = df2.index.map(lambda x: len(list(nx.common_neighbors(G, x[0], x[1]))))

    df2['resource_allocation_index'] =  [i[2] for i in nx.resource_allocation_index(G)]

    df_test2 = df2[df2['Future Connection'].isnull()]
    
    indexes_to_drop = df_test2.index.tolist()
    
    df_train = df2.drop(indexes_to_drop)
    
    y_train = df_train['Future Connection']  
    
    
    X_train = df_train.drop(['Future Connection'], axis=1)
    
#     

    
    clf = RandomForestClassifier().fit(X_train, y_train)
    
    
  
    X_test =  df_test2.fillna(df_test2.mean())
    
   
    X_test = X_test.drop(['Future Connection'], axis=1)
        


    probs =  clf.predict_proba(X_test)[:, 1]
    result = pd.Series(probs, index=X_test.index)
    
#    AUC 0.8305745214752567
    
    return result

new_connections_predictions()

(107, 348)    0.000000
(542, 751)    0.008212
(20, 426)     0.700000
(50, 989)     0.018027
(942, 986)    0.000000
(324, 857)    0.020582
(13, 710)     0.000000
(19, 271)     0.200000
(319, 878)    0.019070
(659, 707)    0.020639
(49, 843)     0.009898
(208, 893)    0.007129
(377, 469)    0.000000
(405, 999)    0.000000
(129, 740)    0.000000
(292, 618)    0.000000
(239, 689)    0.004734
(359, 373)    0.293723
(53, 523)     0.025135
(276, 984)    0.015051
(202, 997)    0.010176
(604, 619)    0.337803
(270, 911)    0.019335
(261, 481)    0.200952
(200, 450)    0.516667
(213, 634)    0.000000
(644, 735)    0.407143
(346, 553)    0.000000
(521, 738)    0.010548
(422, 953)    0.000000
                ...   
(672, 848)    0.019335
(28, 127)     1.000000
(202, 661)    0.009605
(54, 195)     1.000000
(295, 864)    0.006150
(814, 936)    0.014244
(839, 874)    0.011630
(139, 843)    0.016443
(461, 544)    0.015210
(68, 487)     0.000000
(622, 932)    0.022888
(504, 936)    0.090991
(479, 528) 